# Imports

In [ ]:
# !pip install transformers
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

import pandas as pd
import numpy as np

import joblib
import nltk
import re

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Loading the Dataset

In [ ]:
%cd '/content/drive/My Drive/IIITD/SEM-7/ML/ML Project/Code/Dataset'

/content/drive/My Drive/IIITD/SEM-7/ML/ML Project/Code/Dataset


In [ ]:
dataset = pd.read_csv('olid-training-v1.0.tsv', sep='\t', index_col='id')

In [ ]:
test_set = pd.read_csv('testset-levela.tsv', sep='\t', index_col='id')

In [ ]:
subtask_A = dataset[['tweet','subtask_a']]

In [ ]:
subtask_A.head()

,tweet,subtask_a
id,,
86426,@USER She should ask a few native Americans wh...,OFF
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
16820,Amazon is investigating Chinese employees who ...,NOT
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT


# Utility Functions

In [ ]:
def clean_tweet(tweet):
    '''
    removes punctuation marks, other redundant characters, words
    '''
    tweet = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()
    tweet = [word.lower() for word in tweet if word!='URL']
    return ' '.join(tweet)

In [ ]:
# generate columns for extracting features

subtask_A['cleaned'] = subtask_A['tweet'].apply(clean_tweet)
test_set['cleaned'] = test_set['tweet'].apply(clean_tweet)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
subtask_A_cleaned = subtask_A[['cleaned','subtask_a']]
valid_data_cleaned = test_set[['cleaned']]

In [ ]:
subtask_A_cleaned.to_csv('subtask_A_cleaned.csv')
valid_data_cleaned.to_csv('valid_data_cleaned.csv')

# Sentence2Vec [BERT]

In [ ]:
# tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# nlp = transformers.TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model = SentenceTransformer('bert-base-nli-cls-token')

100%|██████████| 405M/405M [00:28<00:00, 14.4MB/s]


In [ ]:
train_sentences = subtask_A['cleaned'].tolist()
test_sentences = test_set['cleaned'].tolist()

train_embeddings = model.encode(train_sentences)
test_embeddings = model.encode(test_sentences)

In [ ]:
joblib.dump(np.array(train_embeddings),'X_train_cls')

['X_train_cls']

In [ ]:
joblib.dump(np.array(test_embeddings),'X_test_cls')

['X_test_cls']

In [ ]:
y_train = np.array(subtask_A['subtask_a'].tolist())
joblib.dump(y_train,'y_train')

['y_train']